In [1]:
import math
from datetime import datetime

import bw2data as bd
import numpy as np
import pandas as pd
import pytest
from bw2data.tests import bw2test
from bw_temporalis import TemporalDistribution

from bw_timex import TimexLCA


@bw2test
def wastes_db():
    bd.projects.set_current("__wastes_db__")
    biosphere = bd.Database("biosphere")
    biosphere.write(
        {
            ("biosphere", "CO2"): {
                "type": "emission",
                "name": "carbon dioxide",
            },
        }
    )

    foreground = bd.Database("foreground")
    foreground.register()
    background_2020 = bd.Database("background_2020")
    background_2020.register()

    node_co2 = biosphere.get("CO2")

    # Create nodes
    fu1 = foreground.new_node("fu1", name="fu1", unit="unit")
    fu1["reference product"] = "fu1"
    fu1.save()

    fu2 = foreground.new_node("fu2", name="fu2", unit="unit")
    fu2["reference product"] = "fu2"
    fu2.save()

    fu3 = foreground.new_node("fu3", name="fu3", unit="unit")
    fu3["reference product"] = "fu3"
    fu3.save()
    
    fu4 = foreground.new_node("fu4", name="fu4", unit="unit")
    fu4["reference product"] = "fu4"
    fu4.save()
    
    fu5 = foreground.new_node("fu5", name="fu5", unit="unit")
    fu5["reference product"] = "fu5"
    fu5.save()

    some_waste_producing_intermediate_process = foreground.new_node(
        "some_waste_producing_intermediate_process", name="some_waste_producing_intermediate_process", unit="unit"
    )
    some_waste_producing_intermediate_process["reference product"] = "some_waste_producing_intermediate_process"
    some_waste_producing_intermediate_process.save()

    other_waste_producing_intermediate_process = foreground.new_node(
        "other_waste_producing_intermediate_process", name="other_waste_producing_intermediate_process", unit="unit"
    )
    other_waste_producing_intermediate_process["reference product"] = "other_waste_producing_intermediate_process"
    other_waste_producing_intermediate_process.save()
    
    some_normal_intermediate_process = foreground.new_node(
        "some_normal_intermediate_process", name="some_normal_intermediate_process", unit="unit"
    )
    some_normal_intermediate_process["reference product"] = "some_normal_intermediate_process"
    some_normal_intermediate_process.save()
    
    used_car = foreground.new_node("used_car", name="used_car", unit="unit")
    used_car["reference product"] = "used_car"
    used_car.save()

    car_treatment_2020 = background_2020.new_node(
        "car_treatment", name="car_treatment", unit="car_treatment"
    )
    car_treatment_2020["reference product"] = "car_treatment"
    car_treatment_2020.save()
    
    some_other_background_process = background_2020.new_node(
        "some_other_background_process", name="some_other_background_process", unit="some_other_background_process"
    )
    some_other_background_process["reference product"] = "some_other_background_process"
    some_other_background_process.save()

    # Edges
    fu1.new_edge(input=fu1, amount=1, type="production").save()
    fu1.new_edge(input=used_car, amount=-1, type="technosphere").save()

    fu2.new_edge(input=fu2, amount=1, type="production").save()
    fu2.new_edge(input=some_waste_producing_intermediate_process, amount=-1, type="technosphere").save()

    fu3.new_edge(input=fu3, amount=1, type="production").save()
    fu3.new_edge(input=car_treatment_2020, amount=-1, type="technosphere").save()
    
    fu4.new_edge(input=fu4, amount=1, type="production").save()
    fu4.new_edge(input=some_normal_intermediate_process, amount=-1, type="technosphere").save()
    
    fu5.new_edge(input=fu5, amount=1, type="production").save()
    fu5.new_edge(input=other_waste_producing_intermediate_process, amount=1, type="technosphere").save()

    some_waste_producing_intermediate_process.new_edge(
        input=some_waste_producing_intermediate_process, amount=1, type="production"
    ).save()
    some_waste_producing_intermediate_process.new_edge(
        input=car_treatment_2020, amount=-3, type="technosphere"
    ).save()
    
    other_waste_producing_intermediate_process.new_edge(
        input=other_waste_producing_intermediate_process, amount=-1, type="production"
    ).save()
    other_waste_producing_intermediate_process.new_edge(
        input=some_waste_producing_intermediate_process, amount=4, type="technosphere"
    ).save()
    
    some_normal_intermediate_process.new_edge(
        input=some_normal_intermediate_process, amount=1, type="production"
    ).save()
    some_normal_intermediate_process.new_edge(
        input=some_other_background_process, amount=3, type="technosphere"
    ).save() 

    used_car.new_edge(input=used_car, amount=-1, type="production").save()
    used_car.new_edge(input=car_treatment_2020, amount=-3, type="technosphere").save()

    car_treatment_2020.new_edge(
        input=car_treatment_2020, amount=-1, type="production"
    ).save()
    car_treatment_2020.new_edge(input=node_co2, amount=1, type="biosphere").save()
    
    some_other_background_process.new_edge(
        input=some_other_background_process, amount=-1, type="production"
    ).save()
    some_other_background_process.new_edge(input=node_co2, amount=1, type="biosphere").save()

    bd.Method(("GWP", "example")).write(
        [
            (("biosphere", "CO2"), 1),
        ]
    )



wastes_db()


Vacuuming database 


In [2]:

method = ("GWP", "example")
database_dates = {
    "background_2020": datetime.strptime("2020", "%Y"),
    "foreground": "dynamic",
}
fu = ("foreground", "fu1")
tlca = TimexLCA({fu: 1}, method, database_dates)
tlca.build_timeline()
tlca.lci()
tlca.static_lcia()

print(tlca.base_lca.score)
print(tlca.static_score)




Starting graph traversal
Calculation count: 2
3.0
-3.0


/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timex_lca.py:194: UserWarning: No edge filter function provided. Skipping all edges within background databases.
  warnings.warn(
/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timeline_builder.py:530: Warning: Reference date 2024-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


In [3]:
tlca.timeline

,hash_producer,time_mapped_producer,date_producer,producer,producer_name,hash_consumer,time_mapped_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,2024,13,2024-01-01,2,fu1,2024,-1,2024-01-01,-1,-1,1.0,None
1,2024,14,2024-01-01,10,used_car,2024,13,2024-01-01,2,fu1,-1.0,None
2,2024,15,2024-01-01,11,car_treatment,2024,14,2024-01-01,10,used_car,3.0,{'background_2020': 1}


In [4]:
method = ("GWP", "example")
database_dates = {
    "background_2020": datetime.strptime("2020", "%Y"),
    "foreground": "dynamic",
}
fu = ("foreground", "fu2")
tlca = TimexLCA({fu: 1}, method, database_dates)
tlca.build_timeline()
tlca.lci()
tlca.static_lcia()


print(tlca.base_lca.score)
print(tlca.static_score)


Starting graph traversal
Calculation count: 2
-3.0
-3.0


/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timex_lca.py:194: UserWarning: No edge filter function provided. Skipping all edges within background databases.
  warnings.warn(
/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timeline_builder.py:530: Warning: Reference date 2024-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


In [5]:
tlca.timeline

,hash_producer,time_mapped_producer,date_producer,producer,producer_name,hash_consumer,time_mapped_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,2024,13,2024-01-01,3,fu2,2024,-1,2024-01-01,-1,-1,1.0,None
1,2024,14,2024-01-01,7,some_waste_producing_intermediate_process,2024,13,2024-01-01,3,fu2,-1.0,None
2,2024,15,2024-01-01,11,car_treatment,2024,14,2024-01-01,7,some_waste_producing_intermediate_process,-3.0,{'background_2020': 1}


In [6]:
method = ("GWP", "example")
database_dates = {
    "background_2020": datetime.strptime("2020", "%Y"),
    "foreground": "dynamic",
}
fu = ("foreground", "fu3")
tlca = TimexLCA({fu: 1}, method, database_dates)
tlca.build_timeline()
tlca.lci()
tlca.static_lcia()


print(tlca.base_lca.score)
print(tlca.static_score)

Starting graph traversal
Calculation count: 1
1.0
1.0


/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timex_lca.py:194: UserWarning: No edge filter function provided. Skipping all edges within background databases.
  warnings.warn(
/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timeline_builder.py:530: Warning: Reference date 2024-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


In [7]:
tlca.timeline

,hash_producer,time_mapped_producer,date_producer,producer,producer_name,hash_consumer,time_mapped_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,2024,13,2024-01-01,4,fu3,2024,-1,2024-01-01,-1,-1,1.0,None
1,2024,14,2024-01-01,11,car_treatment,2024,13,2024-01-01,4,fu3,-1.0,{'background_2020': 1}


In [8]:

method = ("GWP", "example")
database_dates = {
    "background_2020": datetime.strptime("2020", "%Y"),
    "foreground": "dynamic",
}
fu = ("foreground", "fu4")
tlca = TimexLCA({fu: 1}, method, database_dates)
tlca.build_timeline()
tlca.lci()
tlca.static_lcia()

print(tlca.base_lca.score)
print(tlca.static_score)




/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timex_lca.py:194: UserWarning: No edge filter function provided. Skipping all edges within background databases.
  warnings.warn(
/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timeline_builder.py:530: Warning: Reference date 2024-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(


Starting graph traversal
Calculation count: 2
3.0
3.0000000000000004


/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


In [11]:

method = ("GWP", "example")
database_dates = {
    "background_2020": datetime.strptime("2020", "%Y"),
    "foreground": "dynamic",
}
fu = ("foreground", "fu5")
tlca = TimexLCA({fu: 1}, method, database_dates)
tlca.build_timeline()
tlca.lci()
tlca.static_lcia()

print(tlca.base_lca.score)
print(tlca.static_score)


Starting graph traversal
Calculation count: 3
-12.0
12.0


/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timex_lca.py:194: UserWarning: No edge filter function provided. Skipping all edges within background databases.
  warnings.warn(
/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timeline_builder.py:530: Warning: Reference date 2024-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


In [12]:
tlca.timeline

,hash_producer,time_mapped_producer,date_producer,producer,producer_name,hash_consumer,time_mapped_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,2024,13,2024-01-01,6,fu5,2024,-1,2024-01-01,-1,-1,1.0,None
1,2024,14,2024-01-01,7,some_waste_producing_intermediate_process,2024,15,2024-01-01,8,other_waste_producing_intermediate_process,-4.0,None
2,2024,15,2024-01-01,8,other_waste_producing_intermediate_process,2024,13,2024-01-01,6,fu5,1.0,None
3,2024,16,2024-01-01,11,car_treatment,2024,14,2024-01-01,7,some_waste_producing_intermediate_process,-3.0,{'background_2020': 1}


```mermaid
flowchart LR
car_treatment_2020--"-1"-->car_treatment_2020
car_treatment_2020--"-3"-->some_waste_producing_intermediate_process
some_waste_producing_intermediate_process--"1"-->some_waste_producing_intermediate_process
some_waste_producing_intermediate_process--"4"-->other_waste_producing_intermediate_process
other_waste_producing_intermediate_process--"-1"-->other_waste_producing_intermediate_process
other_waste_producing_intermediate_process--"1"-->fu5
fu5--"1"-->fu5
````